In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score

In [17]:
data = pd.read_json('Downloads/politifact_factcheck_data.json', lines=True)
data

,verdict,statement_originator,statement,statement_date,statement_source,factchecker,factcheck_date,factcheck_analysis_link
0,true,Barack Obama,John McCain opposed bankruptcy protections for...,6/11/2008,speech,Adriel Bettelheim,6/16/2008,https://www.politifact.com/factchecks/2008/jun...
1,false,Matt Gaetz,"""Bennie Thompson actively cheer-led riots in t...",6/7/2022,television,Yacob Reyes,6/13/2022,https://www.politifact.com/factchecks/2022/jun...
2,mostly-true,Kelly Ayotte,"Says Maggie Hassan was ""out of state on 30 day...",5/18/2016,news,Clay Wirestone,5/27/2016,https://www.politifact.com/factchecks/2016/may...
3,false,Bloggers,"""BUSTED: CDC Inflated COVID Numbers, Accused o...",2/1/2021,blog,Madison Czopek,2/5/2021,https://www.politifact.com/factchecks/2021/feb...
4,half-true,Bobby Jindal,"""I'm the only (Republican) candidate that has ...",8/30/2015,television,Linda Qiu,8/30/2015,https://www.politifact.com/factchecks/2015/aug...
...,...,...,...,...,...,...,...,...
21147,mostly-false,Donald Trump,Says the large trade deficit with Japan stems ...,8/13/2019,speech,Jon Greenberg,8/15/2019,https://www.politifact.com/factchecks/2019/aug...
21148,false,Donald Trump Jr.,"""Tens of thousands"" of people leave New York e...",11/1/2019,social_media,Jill Terreri Ramos,11/8/2019,https://www.politifact.com/factchecks/2019/nov...
21149,mostly-false,Chris Abele,"""I have fought for our shared values without b...",1/4/2011,news,Dave Umhoefer,1/13/2011,https://www.politifact.com/factchecks/2011/jan...
21150,false,Bloggers,"""Germany halts all Covid-19 vaccines, says the...",8/27/2021,blog,Ciara O'Rourke,9/9/2021,https://www.politifact.com/factchecks/2021/sep...


# Cleaning

In [18]:
data['statement_month'] = pd.to_datetime(data['statement_date']).dt.month
data['statement_year'] = pd.to_datetime(data['statement_date']).dt.year
data['factcheck_month'] = pd.to_datetime(data['factcheck_date']).dt.month
data['factcheck_year'] = pd.to_datetime(data['factcheck_date']).dt.year
data = data.drop(['statement_date', 'factcheck_date'], axis = 1)

In [19]:
data['verdict'].value_counts()

false           5625
half-true       3597
mostly-false    3432
mostly-true     3332
pants-fire      2703
true            2463
Name: verdict, dtype: int64

In [20]:
def verdict_cleaner(verdict):
    if verdict == "mostly-true":
        return True
    elif verdict == "mostly-false":
        return False
    elif verdict == "pants-fire":
        return False
    elif verdict == "half-true":
        return True
    elif verdict == "true":
        return True
    elif verdict == "false":
        return False
data['verdict'] = data['verdict'].apply(lambda x: verdict_cleaner(x))

In [48]:
features = ['statement_originator', 'statement_source', 'factchecker', 'statement_month', 'statement_year', 'factcheck_month', 'factcheck_year']
X = data[features][:gpt_labels.size]
y = data['verdict'].to_numpy()[:gpt_labels.size]

In [49]:
X = pd.get_dummies(X, columns = features).to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,))
clf.fit(X_train, y_train)

/Users/aryamansinha/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(15,), solver='lbfgs')

In [26]:
clf.coefs_

[array([[-0.02006576, -0.00168578, -0.02763794, ..., -0.02205827,
         -0.03367699,  0.02568597],
        [-0.0270251 , -0.00232394, -0.00242388, ..., -0.02984008,
          0.00576328,  0.0097319 ],
        [ 0.07297819,  0.12770793,  0.14547976, ...,  0.05376882,
         -0.02337113, -0.00825092],
        ...,
        [ 0.05953523,  0.57798697, -1.19289254, ...,  0.08613265,
          0.03116958,  0.26115091],
        [ 0.23309602,  0.60212458,  0.34645248, ..., -0.74435919,
         -0.10915374, -0.83851165],
        [-0.3310556 , -0.49906313,  0.01534623, ...,  0.25558392,
          0.01172932,  0.02280451]]), array([[ 7.05303712],
        [ 6.64103525],
        [ 5.87190094],
        [-0.95851829],
        [-3.84137772],
        [-4.78449441],
        [-5.73583833],
        [ 2.59961356],
        [-4.41170045],
        [ 0.51681217],
        [-5.39789371],
        [-2.55537446],
        [ 8.45976051],
        [ 1.29297138],
        [ 3.47177661]])]

In [28]:
clf.intercepts_

[array([-0.66441623,  0.20165708, -0.00965081,  0.06184447,  0.70666166,
         0.08768057, -0.07707376,  1.23139407,  0.61924861, -0.13129165,
        -0.70085283, -0.65181311, -0.57934973, -0.19073765, -0.18667326]),
 array([1.62036359])]

In [27]:
clf.out_activation_

'logistic'

In [ ]:
param_grid = [
    {'solver': ['lbfgs', 'sgd'], 'hidden_layer_sizes':[(15,), (5,), (10,)], 'alpha': [1e-4, 1e-5, 1e-6]}
]
grid_search = GridSearchCV(clf, param_grid, scoring = 'precision')
grid_search.fit(X_train, y_train)

In [27]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_hidden_layer_sizes,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.826738,0.208688,0.004993,0.000248,0.0001,"(15,)",lbfgs,"{'alpha': 0.0001, 'hidden_layer_sizes': (15,),...",0.560000,0.524390,0.483871,0.518987,0.523810,0.522212,0.024148,16
1,3.786520,0.043081,0.002978,0.000342,0.0001,"(15,)",sgd,"{'alpha': 0.0001, 'hidden_layer_sizes': (15,),...",0.750000,0.627119,0.562500,0.551724,0.618182,0.621905,0.070574,3
2,0.811721,0.354861,0.002962,0.000282,0.0001,"(5,)",lbfgs,"{'alpha': 0.0001, 'hidden_layer_sizes': (5,), ...",0.558140,0.609195,0.569767,0.578313,0.578947,0.578873,0.016927,8
3,2.392302,0.096564,0.003941,0.000315,0.0001,"(5,)",sgd,"{'alpha': 0.0001, 'hidden_layer_sizes': (5,), ...",0.781250,0.676471,0.555556,0.580645,0.000000,0.518784,0.271319,17
4,0.990354,0.400955,0.003253,0.000207,0.0001,"(10,)",lbfgs,"{'alpha': 0.0001, 'hidden_layer_sizes': (10,),...",0.578313,0.568182,0.449438,0.540230,0.563218,0.539876,0.046910,13
5,3.323489,0.265982,0.003442,0.000398,0.0001,"(10,)",sgd,"{'alpha': 0.0001, 'hidden_layer_sizes': (10,),...",0.727273,0.636364,0.552239,0.515152,0.600000,0.606205,0.073224,4
6,0.873862,0.078980,0.005329,0.000283,0.00001,"(15,)",lbfgs,"{'alpha': 1e-05, 'hidden_layer_sizes': (15,), ...",0.522727,0.573333,0.511364,0.576471,0.482353,0.533250,0.036483,14
7,5.656241,1.110400,0.004690,0.001572,0.00001,"(15,)",sgd,"{'alpha': 1e-05, 'hidden_layer_sizes': (15,), ...",0.805556,0.636364,0.607143,0.544118,0.527273,0.624090,0.099133,2
8,1.687885,0.755351,0.007261,0.001020,0.00001,"(5,)",lbfgs,"{'alpha': 1e-05, 'hidden_layer_sizes': (5,), '...",0.580247,0.530120,0.568182,0.530864,0.505263,0.542935,0.027418,12
9,5.650121,1.107171,0.007872,0.001410,0.00001,"(5,)",sgd,"{'alpha': 1e-05, 'hidden_layer_sizes': (5,), '...",0.750000,0.596154,0.582090,0.500000,0.540000,0.593649,0.085128,5


In [50]:
clf.score(X_test, y_test)

0.6176853055916776

In [60]:
clf.predict(X_test)

array([False, False, False, ..., False,  True,  True])

In [64]:
tn, fp, fn, tp = confusion_matrix(y_test, clf.predict(X_test)).ravel()

In [59]:
tp / (tp + fn)

0.6016355140186916

In [60]:
fn / (tp + fn)

0.3983644859813084

In [61]:
tn / (tn + fp)

0.6544772620721988

In [62]:
fp / (tn + fp)

0.3455227379278012

In [30]:
minus_250_long = pd.DataFrame(data['statement'].str.len().sort_values()[:20700])
final_data = data.merge(minus_250_long,left_index=True,right_index=True)

In [32]:
gpt_labels = pd.read_csv("Downloads/prediction_results")['0']

0         True
1        False
2         True
3        False
4         True
         ...  
11644     True
11645     True
11646    False
11647    False
11648    False
Name: 0, Length: 11649, dtype: bool

In [63]:
new_X = pd.DataFrame(X).iloc[:gpt_labels.size]
new_X['new_col'] = gpt_labels
X_train, X_test, y_train, y_test = train_test_split(new_X.to_numpy(), y[:gpt_labels.size], test_size=0.33) 

In [64]:
clf.fit(X_train, y_train)

/Users/aryamansinha/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(15,), solver='lbfgs')

In [65]:
clf.score(X_test, y_test)

0.6317295188556566

In [66]:
clf.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, clf.predict(X_test)).ravel()
tp / (tp + fn)

0.6096009253903991

In [67]:
fn / (tp + fn)

0.39039907460960094

In [68]:
tn / (tn + fp)

0.6498109640831758

In [69]:
fp / (tn + fp)

0.3501890359168242